In [1]:
import sys,os
import random
print(sys.path)
if ".." not in sys.path:
    sys.path.insert(0, "..")

['', '/home/hayley/miniconda3/envs/fastai/lib/python36.zip', '/home/hayley/miniconda3/envs/fastai/lib/python3.6', '/home/hayley/miniconda3/envs/fastai/lib/python3.6/lib-dynload', '/home/hayley/miniconda3/envs/fastai/lib/python3.6/site-packages', '/home/hayley/miniconda3/envs/fastai/lib/python3.6/site-packages/defusedxml-0.5.0-py3.6.egg', '/home/hayley/miniconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions', '/home/hayley/.ipython']


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

# sklearn imports
import sklearn
from sklearn.metrics import make_scorer
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict

# sklearn_crfsuite imports
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# pytorch imports 
import torch
import torch.autograd as autograd
from torch.utils.data import Dataset, DataLoader

from torch import Tensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# train logging
import logging
from tqdm import trange

from nlp_utils.model_utils import Params
from nlp_utils.model_evaluate import evaluate
from nlp_utils.data_loader import SentenceDatasetsLoader
# set a random seed
torch.manual_seed(10);

# model saving and inspection
import joblib
import eli5

# auto-reloads
%reload_ext autoreload
%autoreload 2

In [17]:
data_dir = '../data'

In [19]:
params = Params('../data/base_params.json')

In [42]:
loader = SentenceDatasetsLoader(data_dir, 'word2idx.sav', 'tag2idx.sav')
datasets = loader.load_datasets(['train', 'dev', 'testa'])

../data/train_sentences.sav 
 ../data/train_labels.sav
Loaded  train
../data/dev_sentences.sav 
 ../data/dev_labels.sav
Loaded  dev
../data/testa_sentences.sav 
 ../data/testa_labels.sav
Loaded  testa


In [43]:
train, dev, test = datasets['train'], datasets['dev'], datasets['testa']

In [44]:
train_sentences, train_labels = train['data'], train['labels']
train_sentences = np.array(train_sentences)
train_labels = np.array(train_labels)

In [45]:
train_iter = loader.data_iterator(train, params)

In [46]:
sents, labels = next(train_iter)

> /home/hayley/Workspace/Class/IE/HW1/nlp_utils/data_loader.py(97)data_iterator()
-> for i in range( (dataset['size']+1)//params.batch_size ):


(Pdb)  c


> /home/hayley/Workspace/Class/IE/HW1/nlp_utils/data_loader.py(104)data_iterator()
-> idx2word = {i:w for w,i in self.word2idx.items()}


(Pdb)  c


['Jul', '31', ',', '1996', 'Jan', '31', ',', '1996']
[1, 1, 1, 1, 1, 1, 1, 1]
['"', 'I', 'think', 'he', 'deserves', 'to', 'be', 'seeded', 'as', 'everybody', 'else', 'who', 'is', 'in', 'the', 'top', '16', 'deserves', 'to', 'be', 'seeded', ',', '"', 'Stich', 'said', '.']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1]
['Lead', '<UNK>', ',', 'delivered', 'consumer', '4.5', 'to', '6.0', 'cents', '/', 'pound']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['NORTHAMPTON', ',', 'England', '1996-08-30']
[5, 1, 5, 1]
['Alex', 'Corretja', '(', 'Spain', ')', 'vs.', 'Byron', 'Black', '(', 'Zimbabwe', ')']
[3, 4, 1, 5, 1, 1, 3, 4, 1, 5, 1]
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [50]:
sents.size()

torch.Size([5, 26])

In [53]:
sents[0].type()

'torch.LongTensor'

In [56]:
type(sents[0,0].item())

int

In [58]:
sents[0,0].type()

'torch.LongTensor'

## iterator
[source](https://pymbook.readthedocs.io/en/latest/igd.html)

In [60]:
class MyIterator():
    def __init__(self, low, high):
        self.current = low
        self.high = high
        
    def __iter__(self):
        # returns the iterator itself
        # This is used in for...in... statement
        return self
    
    def __next__(self):
        # returns the next element in the iteration
        if self.current <= self.high:
            self.current += 1
            return self.current - 1
        else:
            raise StopIteration
    

In [61]:
myIterator = MyIterator(0,5)
for ele in myIterator:
    print(ele)

0
1
2
3
4
5


Iterator can be used only once, which means after it reaches its end (ie. raised the StopIteration error) once, it will keep raising the same exception.

In [63]:
myIterator.__next__()

StopIteration: 

## Generators
It's an easier way to create iterators using a keyword `yield` from a function.

In [64]:
def myGenerator():
    print('Inside my generator')
    yield 'a'
    yield 'b'
    yield 'c'
myGenerator()

<generator object myGenerator at 0x7f47ab3ee5c8>

In [65]:
g = myGenerator()

In [66]:
g2 = myGenerator()
print(g==g2)
print(g is g2)

False
False


In [67]:
for ele in g:
    print(ele, ' ')

Inside my generator
a  
b  
c  


In [69]:
for ele in g:
    print(ele)

In [72]:
def counter_gen(low, high):
    while low <= high:
        yield low
        low += 1

In [71]:
g2 = counter_gen(0, 5)


In [113]:
class GeneratorObj():
    def __init__(self,low, high):
        self.low = low
        self.high = high
    def __iter__(self):
        counter = self.low
        while counter <= self.high:
            print(counter)
            yield counter
            counter += 1

In [114]:
gobj = GeneratorObj(0,5)


In [135]:
gobj.__iter__()

<generator object GeneratorObj.__iter__ at 0x7f47ac54aeb8>

In [136]:
for num in gobj:
    print(num)

0
0
1
1
2
2
3
3
4
4
5
5


In [137]:
dir(gobj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'high',
 'low']